## Genetic Algorithm for TSP

### General outline:
1. Population Initialization
    - Choose number of individuals the population will consist of. This value is generally kept constant.
2. Selection
    - Using a specific selection method eg. fitness-proportionate, rank-based or tournament select individuals we will use in the next step.
3. Crossover
    - Using a specific crossover method eg. PMX, k-point or uniform perform crossover on the selected parents.
4. Mutation
    - Based on the mutation rate (probability that any individual/gene is changed) apply mutation on population members.
5. Survivor Selection
    - Using a selection method again choose the individuals of the current population that will make up the next generation. 
7. Termination Check
    - When predefined termination criteria are met return the results of the algorithm.


## Apply Principles to TSP:

### What we will need:
- Classes:
    - `Individual, constructor-parameters: n_cities (int), permutation (list), tsplib_problem (tsplib95.models.StandardProblem)`
    - `Population, constructor-parameters: n_cities (int), tsplib_problem (tsplib95.models.StandardProblem), population_size (int), tournament_size (int)`
    - `GeneticAlgorithm: constructor-parameters: tsplib_problem (tsplib95.models.StandardProblem), population_size (int), generations (int), mutation_rate (float), tournament_size (int)`

- Functions:
    - Class:
        - Individual:
            - `fitness(), return: (int)`
            - `mutate(), parameters: mutation_rate (float)`
        - Population:
            - `initialize_population()`
            - `tournament_selection(), return: (Individual) `
            - `pmx(), parameters: parent1 (Individual), parent2 (Individual), return: (Individual)`
            - `create_next_generation(), parameters: mutation_rate (float)`
            - `evaluate_population()`
            - `get_best_individual(), return: (Individual)`
        - GeneticAlgorithm:
            - `run()`

In [11]:
import random
import tsplib95

In [12]:
class Individual:
    def __init__(self, n_cities, permutation, tsplib_problem):
        self.n_cities = n_cities 
        self.permutation = permutation
        self.tsplib_problem = tsplib_problem
        self.total_distance = 0 

        self.fitness()


    def fitness(self):
        total_distance = 0
        for i in range(self.n_cities - 1):
            total_distance += self.tsplib_problem.get_weight(self.permutation[i], self.permutation[i + 1])

        self.total_distance = total_distance

    def mutate(self, mutation_rate):
        if random.random() < mutation_rate:
            i, j = random.sample(range(self.n_cities), 2)
            self.permutation[i], self.permutation[j] = self.permutation[j], self.permutation[i]


In [13]:
class Population:
    def __init__(self, n_cities, tsplib_problem, population_size=100, tournament_size=5):
        self.n_cities = n_cities
        self.tsplib_problem = tsplib_problem
        self.population_size = population_size
        self.tournament_size = tournament_size 
        self.individuals = []

    def initialize_population(self):
        self.individuals = []
        for _ in range(self.population_size):
            permutation = list(self.tsplib_problem.get_nodes())
            random.shuffle(permutation)
            individual = Individual(self.n_cities, permutation, self.tsplib_problem)
            self.individuals.append(individual)
    
    def tournament_selection(self):
        selected = random.sample(self.individuals, self.tournament_size)
        return min(selected, key=lambda ind: ind.total_distance)

    def pmx(self, parent1, parent2):
        start, end = sorted(random.sample(range(self.n_cities), 2))
        child_permutation = [None] * self.n_cities

        child_permutation[start:end] = parent1.permutation[start:end]

        for i in range(self.n_cities):
            if child_permutation[i] is None:
                for city in parent2.permutation:
                    if city not in child_permutation:
                        child_permutation[i] = city
                        break

        return Individual(self.n_cities, child_permutation, self.tsplib_problem)

    def create_next_generation(self, mutation_rate=0.01):
        new_individuals = []
        while len(new_individuals) < self.population_size:
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            child = self.pmx(parent1, parent2)
            child.mutate(mutation_rate)
            new_individuals.append(child)

        self.individuals = new_individuals

    def evaluate_population(self):
        for individual in self.individuals:
            individual.fitness()

    def get_best_individual(self):
        return min(self.individuals, key=lambda ind: ind.total_distance)

In [14]:
class GeneticAlgorithm:
    def __init__(self, tsplib_problem, population_size=100, generations=500, mutation_rate=0.01, tournament_size=5):
        self.tsplib_problem = tsplib_problem
        self.population_size = population_size
        self.generations = generations
        self.epochs = 0
        self.mutation_rate = mutation_rate
        self.tournament_size = tournament_size
        self.n_cities = len(list(tsplib_problem.get_nodes()))
        self.population = Population(self.n_cities, tsplib_problem, population_size, tournament_size)

    def run(self):
        self.population.initialize_population()
        for generation in range(self.generations):
            self.population.evaluate_population()
            best_individual = self.population.get_best_individual()
            print(f"Generation {generation}: Best Distance = {best_individual.total_distance}")
            self.population.create_next_generation(self.mutation_rate)
            self.epochs += 1

        return self.population.get_best_individual()

In [15]:
if __name__ == "__main__":
    problem = tsplib95.load("data/berlin52.tsp") 
    ga = GeneticAlgorithm(problem, population_size=100, generations=500, mutation_rate=0.1, tournament_size=5)
    best_solution = ga.run()
    print(f"Best solution found: {best_solution.permutation} with distance {best_solution.total_distance}")

Generation 0: Best Distance = 25669
Generation 1: Best Distance = 24737
Generation 2: Best Distance = 23576
Generation 3: Best Distance = 22955
Generation 4: Best Distance = 21507
Generation 5: Best Distance = 21367
Generation 6: Best Distance = 20906
Generation 7: Best Distance = 20712
Generation 8: Best Distance = 20254
Generation 9: Best Distance = 19994
Generation 10: Best Distance = 18338
Generation 11: Best Distance = 18281
Generation 12: Best Distance = 18138
Generation 13: Best Distance = 17250
Generation 14: Best Distance = 17105
Generation 15: Best Distance = 16879
Generation 16: Best Distance = 16617
Generation 17: Best Distance = 16357
Generation 18: Best Distance = 15922
Generation 19: Best Distance = 15887
Generation 20: Best Distance = 15598
Generation 21: Best Distance = 15567
Generation 22: Best Distance = 15225
Generation 23: Best Distance = 15448
Generation 24: Best Distance = 15411
Generation 25: Best Distance = 15269
Generation 26: Best Distance = 15007
Generation 